In [1]:
import pandas as pd
import warnings
import re
import numpy as np
import time
import PyPDF2
from PyPDF2 import PdfReader
import tabula
import os
import shutil

In [2]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'
warnings.simplefilter(action='ignore', category=FutureWarning)
term_size = os.get_terminal_size()

### Select Scouts

In [3]:
file = pd.read_csv("Turn Data/Turn Data.csv").Filepath.squeeze() # get file path 

In [4]:
def check_if_you_scouted(file):
    pdf_reader = PyPDF2.PdfReader(file)
    search_string = 'Scouted Teams' 
    for page_number in range(len(pdf_reader.pages)): 
            text = pdf_reader.pages[page_number].extract_text() 
            if search_string in text: 
                return "Yes"

check_if_you_scouted(file)

'Yes'

In [5]:
def length(file):
    pdf_reader = PyPDF2.PdfReader(file)
    print(len(pdf_reader.pages))
length(file)

55


In [6]:
def find_section_page_number(pdf_path, section_title):
    with open(pdf_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text()
            if text:
                lines = text.split('\n')
                for line in lines:
                    if section_title.lower() in line.lower():
                        # Try to find the page number at the end of the line
                        words = line.split()
                        # Look for the last element in the line that is a digit (likely the page number)
                        for word in reversed(words):
                            if word.isdigit():
                                return int(word) + 3
    return None

def print_page_content(pdf_path, page_number):
    with open(pdf_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        if page_number <= len(reader.pages):
            page = reader.pages[page_number + 3]  # Pages are zero-indexed in PyPDF2
            text = page.extract_text()
            if text:
                print(f"Content of page {page_number}:\n")
                print(text)
            else:
                print(f"No text found on page {page_number}.")
        else:
            print(f"Page {page_number} does not exist in the document.")

def length(file):
    pdf_reader = PyPDF2.PdfReader(file)
    return len(pdf_reader.pages)
length(file)

# =============================

# Example usage
section_title = 'Team of the week'
TotW = find_section_page_number(file, section_title)
scouts_begin_here = TotW+1
pages = length(file)

if scouts_begin_here:
    print(f'{section_title} is on page {TotW}.')
    print(f'Scouts begin at page {int(scouts_begin_here)} of {pages}')
    #print_page_content(file, scouts_begin_here)
else:
    print(f"'{section_title}' not found in the document.")

Team of the week is on page 33.
Scouts begin at page 34 of 55


In [7]:
splicer = []

def page(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(reader.pages)):
            page = page_num + 1
            splicer.append(page)
page(file)


In [8]:
pages_likely_with_scouts = splicer[scouts_begin_here:]
scouts = []
scouted = []

def print_page_content(i,file):
    with open(file, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        try:
            page = reader.pages[i]  # Pages are zero-indexed in PyPDF2
            text = page.extract_text()
            if "Next Weeks " in text:
                print(f"*** {text.partition('\n')[0]}detected on page {i} ***\n")
                pass

            elif "National News" in text:
                pass

            elif "Team of the week" in text:
                pass

            elif "Manager Decision Sheet" in text:
                print(f"MDS sheet detected on page {i}\n")
                pass

            elif len(text) <5:
                pass
                
            elif text:    
                print(f"Scouted team {text.partition('\n')[0]}detected on page {i}:\n")
                scouts.append(i+1)
                scouted.append(text.partition('\n')[0])
                
            else:
                print(f"No text found on page {i}.")
        except: 
            pass

for i in pages_likely_with_scouts:
    #print(i)
    print_page_content(i,file)

print(scouted,scouts)

Scouted team H Arango (Fortaleza) T Ortega (Independ'nte) detected on page 36:

Scouted team Beracruz detected on page 37:

Scouted team Brujas FC detected on page 38:

Scouted team Real Cartagena detected on page 39:

Scouted team Carmelita detected on page 40:

Scouted team Cucuta Deportivo detected on page 41:

*** Next Weeks Opponents - Danubio detected on page 42 ***

Scouted team Atletico Huila detected on page 43:

Scouted team Deportes Quindio detected on page 44:

Scouted team Slovan Liberec detected on page 45:

Scouted team Tamarindo detected on page 46:

Scouted team Unicosta detected on page 47:

*** Next Weeks Reserve Opponents - Danubio detected on page 48 ***

*** Next Weeks Youth Opponents - Danubio detected on page 49 ***

MDS sheet detected on page 50

MDS sheet detected on page 51

MDS sheet detected on page 52

MDS sheet detected on page 53

["H Arango (Fortaleza) T Ortega (Independ'nte) ", 'Beracruz ', 'Brujas FC ', 'Real Cartagena ', 'Carmelita ', 'Cucuta Deporti

In [10]:

# Load the file path
file = pd.read_csv("Turn Data/Turn Data.csv").Filepath.squeeze()  

s_PV = 1  # Example value for PV threshold

def search_scouted(page,teamname):
    try:
        # Open the PDF file
        with open(file, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)  # Create a PDF reader object
            
            # Read the specified page
            pages = tabula.read_pdf(file, pages=page)

            if not pages:
                #print(f"No data found on page {page}.")
                return

            st = pd.DataFrame(pages[0]).replace(np.nan, '')

            if st.iloc[:1, :1].squeeze() == "Goalkeepers":  # Ensure what we're selecting is an MDS team sheet, not something else 
                # --------- filter and apply ---------- #
                print('\033[1m\n')
                print('=' * 50)  # Replace term_size.columns with a fixed size for demonstration
                print('\n',teamname)
                st.drop(list(st.filter(regex='Unnamed: 0')), axis=1, inplace=True)
                st.rename(columns={"Unnamed: 1": "Nationality"}, inplace=True)
                print(f"\033[1m\n")
                print('=' * 50)

                # --------- strip special characters ---------- #
                st.iloc[:, 3:16] = st.iloc[:, 3:16].map(lambda x: x.rstrip('*+-')) 

                # --------- break into groups ---------- #
                s = st.eq(st.columns)
                s_groups = [g.reset_index(drop=True) for _, g in st[~s.iloc[:, 0]].groupby(s.cumsum()[~s.iloc[:, 0]].iloc[:, 0])]
                s_gks = s_groups[0].drop(index=0)
                s_deff = s_groups[1].drop(index=0)
                s_mid = s_groups[2].drop(index=0)
                s_att = s_groups[3].drop(index=0)

                # --------- rename cols ---------- #
                s_deff = s_deff.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref':'Jud', 'Crs':'Vis'})
                s_mid = s_mid.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref':'Jud', 'Crs':'Vis'})
                s_att = s_att.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref':'Jud', 'Crs':'Vis'})

                # --------- split ID and names ---------- #
                s_gks[['ID', 'Name']] = s_gks['ID Name'].str.split(' ', n=1, expand=True)
                s_gks.drop(columns="ID Name", axis=1, inplace=True)
                s_gks = s_gks[["ID", "Name", "Age", "Foot", "Sho", "Mov", "Pas", "Con", "Spe", "Sta", "Hea", "Str", "Dis", "Han", "Ref", "Crs", "OA", "SA", "Nationality", "Price"]]

                # --------- Calculate PV ---------- #
                s_gks['GK'] = s_gks[['Dis', 'Han', 'Ref', 'Crs']].astype(int).sum(axis=1, skipna=True)
                s_deff['CB'] = s_deff[['Hea', 'Str', 'Tac', 'Jud']].astype(int).sum(axis=1, skipna=True)
                s_deff['FB'] = s_deff[['Spe', 'Sta', 'Agg', 'Tac']].astype(int).sum(axis=1, skipna=True)
                s_deff['SW'] = s_deff[['Pas', 'Con', 'Tac', 'Jud']].astype(int).sum(axis=1, skipna=True)
                s_deff['WB'] = s_deff[['Mov', 'Pas', 'Spe', 'Sta']].astype(int).sum(axis=1, skipna=True)

                s_mid['CM'] = s_mid[['Pas', 'Sta', 'Hea', 'Tac']].astype(int).sum(axis=1, skipna=True)
                s_mid['AM'] = s_mid[['Str', 'Agg', 'Tac', 'Jud']].astype(int).sum(axis=1, skipna=True)
                s_mid['PL'] = s_mid[['Pas', 'Con', 'Jud', 'Vis']].astype(int).sum(axis=1, skipna=True)
                s_mid['FR'] = s_mid[['Sho', 'Mov', 'Pas', 'Con']].astype(int).sum(axis=1, skipna=True)
                s_mid['WG'] = s_mid[['Pas', 'Con', 'Spe', 'Sta']].astype(int).sum(axis=1, skipna=True)

                s_att['WF'] = s_att[['Sho', 'Mov', 'Con', 'Spe']].astype(int).sum(axis=1, skipna=True)
                s_att['IF'] = s_att[['Pas', 'Con', 'Jud', 'Vis']].astype(int).sum(axis=1, skipna=True)
                s_att['CF'] = s_att[['Sho', 'Mov', 'Str', 'Agg']].astype(int).sum(axis=1, skipna=True)
                s_att['TM'] = s_att[['Con', 'Hea', 'Str', 'Agg']].astype(int).sum(axis=1, skipna=True)

                # --------- don't show low PVs ---------- #
                s_gks['GK'] = s_gks['GK'].astype(int)
                s_gks['GK'] = np.where(s_gks['GK'] < s_PV, "", s_gks['GK'])

                s_deff['CB'] = s_deff['CB'].astype(int)
                s_deff['CB'] = np.where(s_deff['CB'] < s_PV, "", s_deff['CB'])
                s_deff['SW'] = s_deff['SW'].astype(int)
                s_deff['SW'] = np.where(s_deff['SW'] < s_PV, "", s_deff['SW'])
                s_deff['WB'] = s_deff['WB'].astype(int)
                s_deff['WB'] = np.where(s_deff['WB'] < s_PV, "", s_deff['WB'])
                s_deff['FB'] = s_deff['FB'].astype(int)
                s_deff['FB'] = np.where(s_deff['FB'] < s_PV, "", s_deff['FB'])

                s_mid['CM'] = s_mid['CM'].astype(int)
                s_mid['CM'] = np.where(s_mid['CM'] < s_PV, "", s_mid['CM'])
                s_mid['AM'] = np.where(s_mid['AM'] < s_PV, "", s_mid['AM'])
                s_mid['PL'] = np.where(s_mid['PL'] < s_PV, "", s_mid['PL'])
                s_mid['FR'] = np.where(s_mid['FR'] < s_PV, "", s_mid['FR'])
                s_mid['WG'] = s_mid['WG'].astype(int)
                s_mid['WG'] = np.where(s_mid['WG'] < s_PV, "", s_mid['WG'])

                s_att['WF'] = s_att['WF'].astype(int)
                s_att['WF'] = np.where(s_att['WF'] < s_PV, "", s_att['WF'])
                s_att['IF'] = np.where(s_att['IF'] < s_PV, "", s_att['IF'])
                s_att['TM'] = np.where(s_att['TM'] < s_PV, "", s_att['TM'])
                s_att['CF'] = np.where(s_att['CF'] < s_PV, "", s_att['CF'])

                # --------- Show Players by Section ----------
                display("Goalkeepers", s_gks.style.background_gradient(axis=None, subset=['Spe', 'Dis', 'Crs']).background_gradient(axis=None, vmin=1, vmax=120, cmap="viridis", subset=["OA"]),
                        "Defenders", s_deff.style.background_gradient(axis=None, subset=['Spe', 'Agg', 'Vis']).background_gradient(axis=None, vmin=1, vmax=120, cmap="viridis",subset=["OA"]),
                        "Midfielders", s_mid.style.background_gradient(axis=None, subset=['Spe', 'Agg', 'Vis']).background_gradient(axis=None, vmin=1, vmax=120, cmap="viridis",subset=["OA"]),
                        "Attackers", s_att.style.background_gradient(axis=None, subset=['Spe', 'Agg', 'Vis']).background_gradient(axis=None, vmin=1, vmax=120, cmap="viridis",subset=["OA"])
                       )
                #display("Goalkeepers", s_gks.style.background_gradient(axis=None, cmap="viridis",subset=['GK']).background_gradient(axis=None, vmin=0, vmax=10,subset=["Spe","Dis",'Crs']),
                        #"Defenders", s_deff.style.background_gradient(axis=None, cmap="viridis",subset=['CB', 'FB', 'SW', 'WB']).background_gradient(axis=None, vmin=0, vmax=10, subset=["Spe",'Agg',"Vis"]),
                        #"Midfielders", s_mid.style.background_gradient(axis=None, cmap="viridis", subset=['CM', 'AM', 'PL', "FR", "WG"]).background_gradient(axis=None,vmin=0, vmax=10,  subset=["Spe",'Agg',"Vis"]),
                        #"Attackers", s_att.style.background_gradient(axis=None, cmap="viridis", subset=['WF', 'IF', 'CF', "TM"]).background_gradient(axis=None, vmin=0, vmax=10, subset=["Spe",'Agg',"Vis"])
                       #)
    except Exception as e:
        print(f"An error occurred: {e}")

# Iterate through each element in scouts
for i,j in zip(scouts,scouted):
    #print(i)
    search_scouted(i,j)  # Pass each element of scouts and the page number





 Beracruz 




'Goalkeepers'

,ID,Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Dis,Han,Ref,Crs,OA,SA,Nationality,Price,GK
1,PREE,E Preciado,19,R,4,6,8,7,6,4,4,3,9,3,9,8,71,Inx,Columbian,Free,29
2,GALP,P Galeano,19,R,4,4,5,6,8,5,4,5,7,2,4,5,59,Inx,Columbian,Free,18
3,CCNG,K Caniggia,19,L,3,3,5,4,8,4,3,4,8,2,5,6,55,Inx,Columbian,Free,21


'Defenders'

,ID Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Agg,Tac,Jud,Vis,OA,SA,Nationality,Price,CB,FB,SW,WB
1,CXYP C Cardona,24,R,4,5,7,7,6,6,9,7,7,9,8,7,82,Inx/Tea,Columbian,"£210,000",33,28,31,24
2,CALR R Cordoba,22,L,4,6,7,7,8,5,5,5,8,5,6,9,75,Inx,Columbian,"£4,000",21,26,25,26
3,BOLK K Bolano,18,R,4,3,4,4,7,4,5,6,6,3,3,7,56,Inx,Columbian,Free,17,20,14,18
4,MFHK K Mondragon,17,R,3,3,4,3,7,4,5,6,5,3,2,5,50,Inx,Columbian,Free,16,19,12,18
5,RPRS O Ricard,18,R,3,3,5,5,5,2,3,4,6,2,4,6,48,Inx,Columbian,Free,13,15,16,15
6,VALK K Valderrama,19,R,4,3,3,4,3,2,4,5,5,4,2,6,45,Inx,Columbian,Free,15,14,13,11


...

,ID Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Agg,Tac,Jud,Vis,OA,SA,Nationality,Price,CB,FB,SW,WB
1,MONF F Mozer,19,R,6,6,8,9,10,8,6,9,5,6,6,10,89,Inx,Columbian,"£306,000",27,29,29,32
2,TQBP C Terán,23,R,5,6,7,7,6,6,9,8,7,9,8,8,86,,Columbian,"£4,100,000",34,28,31,25
3,ABOC S Asprilla,23,L,4,4,5,6,6,4,5,5,7,5,5,8,64,Inx,Columbian,Free,20,22,21,19
4,MBKX T Morates,18,R,3,4,5,5,7,4,4,6,6,5,3,6,58,Inx,Columbian,Free,18,22,18,20
5,RICE E Ricard,19,R,3,6,4,4,7,4,5,4,4,4,3,7,55,Inx,Columbian,Free,16,19,15,21
6,PEPC C Pepelu,20,L,3,3,4,5,6,3,6,5,7,1,3,5,51,Inx,Columbian,Free,15,17,13,16


'Midfielders'

,ID Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Agg,Tac,Jud,Vis,OA,SA,Nationality,Price,CM,AM,PL,FR,WG
1,SDOW S Serna,21,R,4,4,5,5,8,6,8,9,7,7,9,8,80,Def,Columbian,"£132,000",26,32,27,18,24
2,QUIO O Quinones,19,R,5,4,7,7,7,3,6,5,8,2,2,9,65,Inx,Columbian,Free,18,17,25,23,24
3,ZULN N Zuleta,18,R,3,6,4,5,4,3,4,4,7,2,4,7,53,Inx,Columbian,Free,13,17,20,18,16
4,VTOB G Valderama,19,R,2,6,4,4,4,3,5,4,4,4,3,6,49,Inx,Columbian,Free,16,15,17,16,15


'Attackers'

,ID Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Agg,Tac,Jud,Vis,OA,SA,Nationality,Price,WF,IF,CF,TM
1,ARBN N Arboleda,24,R,4,5,6,6,7,4,6,5,7,6,8,7,71,,Columbian,Free,22,27,21,24
2,PKML O Pacheco,19,L,4,3,4,4,5,2,4,7,8,3,2,7,53,Inx,Columbian,Free,16,17,22,23
3,GMPK K Gaviria,19,B,3,3,5,4,5,3,4,4,6,3,4,7,51,Inx,Columbian,Free,15,20,16,18
